#Transmitter side

In [2]:
## Hamming 16 bit -Transmitter " Standard 7a"
import numpy as np
input_data = "4"
print("input data:", input_data)
# 1. Convert Teks ke Biner (Bit Stream)
def text_to_bits(text):
    return ''.join(format(ord(char), '08b') for char in text)

def hamming_encoder(data_bits):
    encoded_bits = []
    for i in range(0, len(data_bits), 4):
        # Ambil 4 bit data
        data = data_bits[i:i+4]
        
        # Hitung bit paritas
        p1 = (int(data[0]) + int(data[1]) + int(data[3])) % 2  # p1 = d1 + d2 + d4
        p2 = (int(data[0]) + int(data[2]) + int(data[3])) % 2  # p2 = d1 + d3 + d4
        p3 = (int(data[1]) + int(data[2]) + int(data[3])) % 2  # p3 = d2 + d3 + d4
        p4 = (p1 + p2 + int(data[0]) + p3 + int(data[1]) + int(data[2]) + int(data[3])) % 2  # p4 = p1 + p2 + d1 + p3 + d2 + d3 + d4
        
        # Gabungkan dalam urutan Hamming (8,4): p1, p2, d1, p3, d2, d3, d4, p4
        encoded_bits.append(str(p1) + str(p2) + data[0] + str(p3) + data[1] + data[2] + data[3] + str(p4))
    
    return ''.join(encoded_bits)

# 3. RLL Encoder (Manchester)
def manchester_encoding(data_bits):
    manchester_bits = []
    for bit in data_bits:
        if bit == '1':
            manchester_bits.append('10')  # Manchester encoding untuk 1
        else:
            manchester_bits.append('01')  # Manchester encoding untuk 0
    return ''.join(manchester_bits)

# 4. Low Rate Preamble (6 bit)
def add_preamble(encoded_bits, preamble="101010"):
    return preamble + encoded_bits

# 5. OOK Modulation
def ook_modulation(data_bits):
    # OOK: 1 -> ON, 0 -> OFF
    return [1 if bit == '1' else 0 for bit in data_bits]

# Main Transmitter Function
def transmitter(input_data):
    # Step 1: Convert Teks ke Biner (Bit Stream)
    bit_stream = text_to_bits(input_data)
    print(f"Bit Stream: {bit_stream}")
    
    # Step 2: FEC Encoding (Hamming 8,4)
    encoded_bits = hamming_encoder(bit_stream)
    print(f"Encoded Bits (Hamming 8,4): {encoded_bits}")
    
    # Step 3: RLL Encoding (Manchester)
    manchester_bits = manchester_encoding(encoded_bits)
    print(f"Manchester Encoded Bits: {manchester_bits}")
    
    # Step 4: Add Low Rate Preamble (6-bit)
    preambled_bits = add_preamble(manchester_bits)
    print(f"Bits with Preamble: {preambled_bits}")
    
    # Step 5: OOK Modulation
    modulated_signal = ook_modulation(preambled_bits)
    print(f"OOK Modulated Signal: {modulated_signal}")
    
    return modulated_signal


modulated_signal = transmitter(input_data)


input data: 4
Bit Stream: 00110100
Encoded Bits (Hamming 8,4): 1000011110011001
Manchester Encoded Bits: 10010101011010101001011010010110
Bits with Preamble: 10101010010101011010101001011010010110
OOK Modulated Signal: [1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0]


#Reciver side

In [5]:

## Hamming 16 bit -Receiver " Standard 7a"


modulated_signal = [1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0]
print("OOK Demodulated Signal", modulated_signal)

# 1. OOK Demodulation
def ook_demodulation(modulated_signal):
    # Mengubah sinyal OOK kembali menjadi bit stream: 1 -> ON, 0 -> OFF
    return ''.join('1' if bit == 1 else '0' for bit in modulated_signal)

# 2. Remove Preamble
def remove_preamble(encoded_bits, preamble="101010"):
    # Menghapus preamble yang ditambahkan sebelumnya
    return encoded_bits[len(preamble):]

# 3. RLL Decoding (Manchester)
def manchester_decoding(data_bits):
    decoded_bits = []
    # Setiap dua bit pada data adalah hasil Manchester encoding
    for i in range(0, len(data_bits), 2):
        if data_bits[i:i+2] == "10":
            decoded_bits.append('1')
        elif data_bits[i:i+2] == "01":
            decoded_bits.append('0')
    return ''.join(decoded_bits)

# 4. FEC Decoder (Hamming 8,4)
def hamming_decoder(encoded_bits):
    decoded_bits = []
    for i in range(0, len(encoded_bits), 8):
        # Ambil grup 8 bit yang sudah diterima
        group = encoded_bits[i:i+8]
        p1, p2, d1, p3, d2, d3, d4, p4 = group
        
        # Hitung syndrome untuk perbaikan bit error
        s1 = (int(d1) + int(d2) + int(d4) + int(p1)) % 2
        s2 = (int(d1) + int(d3) + int(d4) + int(p2)) % 2
        s3 = (int(d2) + int(d3) + int(d4) + int(p3)) % 2
        
        # Tentukan posisi bit yang error
        error_pos = s1 * 1 + s2 * 2 + s3 * 4
        
        if error_pos != 0:
            # Perbaiki bit yang error (posisi dihitung dari 1)
            error_pos -= 1  # Mengubah ke index 0
            group = list(group)
            group[error_pos] = '1' if group[error_pos] == '0' else '0'  # Membalikkan bit error

        # Ambil data yang terdekode (d1, d2, d3, d4)
        decoded_bits.append(group[2])
        decoded_bits.append(group[4])
        decoded_bits.append(group[5])
        decoded_bits.append(group[6])
    
    return ''.join(decoded_bits)

# Fungsi untuk mengubah biner menjadi karakter yang dapat dibaca
def binary_to_text(binary_string):
    # Pecah string biner menjadi blok 8-bit dan konversi setiap blok ke karakter
    n = 8  # Setiap karakter adalah 8-bit
    text = ''
    for i in range(0, len(binary_string), n):
        byte = binary_string[i:i+n]
        text += chr(int(byte, 2))  # Konversi byte biner ke karakter
    return text


# Main Receiver Function
def receiver(modulated_signal):
    # Step 1: OOK Demodulation
    demodulated_signal = ook_demodulation(modulated_signal)
    print(f"Demodulated Signal: {demodulated_signal}")
    
    # Step 2: Remove Preamble
    signal_without_preamble = remove_preamble(demodulated_signal)
    print(f"Manchested Decoded Bits: {signal_without_preamble}")  ###Signal without Preamble
    
    # Step 3: RLL Decoding (Manchester)
    manchester_decoded_bits = manchester_decoding(signal_without_preamble)
    print(f"Decoded Bits (Hamming 8,4): : {manchester_decoded_bits}") ##Manchester Decoded Bits
    
    # Step 4: FEC Decoding (Hamming 8,4)
    decoded_bits = hamming_decoder(manchester_decoded_bits)
    print(f"Bit Stream: {decoded_bits}") #Decoded Bits (After Hamming)
    

    # Convert decoded bits to readable text
    readable_text = binary_to_text(decoded_bits)
    print(f"Readable Text: {readable_text}")

    return decoded_bits

# Call the Receiver function
decoded_bits = receiver(modulated_signal)


OOK Demodulated Signal [1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0]
Demodulated Signal: 10101010010101011010101001011010010110
Manchested Decoded Bits: 10010101011010101001011010010110
Decoded Bits (Hamming 8,4): : 1000011110011001
Bit Stream: 00110100
Readable Text: 4
